In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

Now here we are connecting with database to connect it we are using variable con

In [2]:
con=sqlite3.connect("database.sqlite")

In [3]:
filtered_data=pd.read_sql_query("SELECT * from Reviews where Score!=3 LIMIT 5000",con)

Now we will remove the data of Score with 1 and 0 here 4,5 means->1 and 1,2 ->0 and 3 we dont know

In [4]:
def partition(x):
    if x < 3:
        return 0
    return 1
actualScore=filtered_data["Score"]
positiveNegative=actualScore.map(partition)
filtered_data["Score"]=positiveNegative
print(filtered_data.shape)
filtered_data.head()

(5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
filtered_data['Score'].value_counts()

1    4187
0     813
Name: Score, dtype: int64

In [6]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
display=pd.read_sql_query("""
    SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
    From Reviews group by UserId having count(*)>1""",con)
                          

In [8]:
print(display.shape)
display.head()

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [9]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [10]:
display['COUNT(*)'].sum()

393063

From here we can see that there are various duplicates and we are removing it.

# Exploratory Data Analysis
Data Cleaning: Deduplication

In [11]:
sorted_values=filtered_data.sort_values("ProductId",axis=0,ascending=True,inplace=False,kind='quicksort',na_position='last')

In [12]:
final=sorted_values.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},inplace=False,keep='first')
print(final.size,final.shape)

49860 (4986, 10)


In [13]:
print((final['Id'].size*1.0/filtered_data['Id'].size*1.0)*100)

99.72


In [14]:
final.shape

(4986, 10)

In [15]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND HelpfulnessNumerator>HelpfulnessDenominator
ORDER BY ProductID
""", con)

In [16]:
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [17]:
final=final[final["HelpfulnessNumerator"]<=final["HelpfulnessDenominator"]]


In [18]:
final['Score'].value_counts()

1    4178
0     808
Name: Score, dtype: int64

# Text preprocessing
1.Begin by removing the html tags
2.Remove any punctuations or limited set of special characters like , or . or # etc.
3.Check if the word is made up of english letters and is not alpha-numeric
4.Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5.Convert the word to lowercase
6.Remove Stopwords
7.Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)

In [19]:
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
Wow.  So far, two two-star reviews.  One obviously had no 

In [20]:
sent_0=re.sub(r"http\S+", "", sent_0)
sent_1000=re.sub(r"http\S+", "", sent_1000)
sent_1500=re.sub(r"http\S+", "", sent_1500)
sent_4900=re.sub(r"http\S+", "", sent_4900)

In [21]:
sent_0

'Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.'

In [22]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

Why is this $[...] when the same product is available for $[...] here? />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I recently tried this flavor/brand and was surprised at how delicious these chips are.  The best thing was that there were a lot of "brown" chips in the bsg (my favorite), so I bought some more through amazon and shared with family and friends.  I am a little disappointed that there are not, so far, very many brown chips in these bags, but the flavor is still very good.  I like them better than the yogurt and green onion flavor because they do not seem to be as salty, and the onion flavor is better.  If you haven't eaten Kettle chips before, I recommend that you try a bag before buying bulk.  They are thicker and crunchier than Lays but just as fresh out of the bag.
Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I'm sorry; b

In [23]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [24]:
sent_0=decontracted(sent_0)
sent_1000=decontracted(sent_1000)
sent_1500=decontracted(sent_1500)
sent_4900=decontracted(sent_4900)

In [25]:
sent_1500

'Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I am sorry; but these reviews do nobody any good beyond reminding us to look  before ordering.<br /><br />These are chocolate-oatmeal cookies.  If you do not like that combination, do not order this type of cookie.  I find the combo quite nice, really.  The oatmeal sort of "calms" the rich chocolate flavor and gives the cookie sort of a coconut-type consistency.  Now let is also remember that tastes differ; so, I have given my opinion.<br /><br />Then, these are soft, chewy cookies -- as advertised.  They are not "crispy" cookies, or the blurb would say "crispy," rather than "chewy."  I happen to like raw cookie dough; however, I do not see where these taste like raw cookie dough.  Both are soft, however, so is this the confusion?  And, yes, they stick together.  Soft cookies tend to do that.  They are not individually wrapped, which would add to the cost.  Oh 

In [26]:
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
sent_1000 = re.sub("\S*\d\S*", "", sent_1000).strip()
sent_1500 = re.sub("\S*\d\S*", "", sent_1500).strip()
sent_4900 = re.sub("\S*\d\S*", "", sent_4900).strip()
print(sent_4900)

love to order my coffee on amazon.  easy and shows up quickly.<br />This k cup is great coffee.  dcaf is very good as well


In [27]:
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
sent_1000 = re.sub('[^A-Za-z0-9]+', ' ', sent_1000)
sent_0 = re.sub('[^A-Za-z0-9]+', ' ', sent_0)
sent_4900 = re.sub('[^A-Za-z0-9]+', ' ', sent_4900)
print(sent_1500)

Wow So far two two star reviews One obviously had no idea what they were ordering the other wants crispy cookies Hey I am sorry but these reviews do nobody any good beyond reminding us to look before ordering br br These are chocolate oatmeal cookies If you do not like that combination do not order this type of cookie I find the combo quite nice really The oatmeal sort of calms the rich chocolate flavor and gives the cookie sort of a coconut type consistency Now let is also remember that tastes differ so I have given my opinion br br Then these are soft chewy cookies as advertised They are not crispy cookies or the blurb would say crispy rather than chewy I happen to like raw cookie dough however I do not see where these taste like raw cookie dough Both are soft however so is this the confusion And yes they stick together Soft cookies tend to do that They are not individually wrapped which would add to the cost Oh yeah chocolate chip cookies tend to be somewhat sweet br br So if you wa

In [28]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [29]:
from tqdm import tqdm
preprocessed_reviews=[]
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|█████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:06<00:00, 748.77it/s]


In [30]:
preprocessed_reviews[300]

'ordered product dog loves no one cares degrees outside recieved one big melted ball thought would ship dry ice summer hot area dont order'

# Bag of Words

In [31]:
count_vector=CountVectorizer()
count_vector.fit(preprocessed_reviews)
print(count_vector.get_feature_names()[0:11])
print("="*50)
final_counts=count_vector.transform(preprocessed_reviews)
final_counts.shape
final_counts.get_shape()[1]

['aa', 'aahhhs', 'aback', 'abandon', 'abates', 'abbott', 'abby', 'abdominal', 'abiding', 'ability', 'able']


12997

# N-grams

In [32]:
count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
final_bigram_counts = count_vect.fit_transform(preprocessed_reviews)
print(final_bigram_counts.toarray())
final_bigram_counts.get_shape()


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(4986, 3144)

# Self-explanation of tf-idf

In [33]:
tfidf_vect=TfidfVectorizer()
aman=['This is a Sentence','this is a Aman Stark',"This boy is billionarie"]
x=tfidf_vect.fit(aman)
#x.vocabulary_
tfidf_vect.get_feature_names()
x=tfidf_vect.transform(aman)
#x.toarray()
dd=pd.DataFrame(x.toarray(),columns=tfidf_vect.get_feature_names())
dd

,aman,billionarie,boy,is,sentence,stark,this
0,0.000000,0.000000,0.000000,0.453295,0.767495,0.000000,0.453295
1,0.608845,0.000000,0.000000,0.359594,0.000000,0.608845,0.359594
2,0.000000,0.608845,0.608845,0.359594,0.000000,0.000000,0.359594


# Main preproceesed Tfidf

In [34]:
tfidf_vector=TfidfVectorizer()
tfidf_vector.fit(preprocessed_reviews)
tfidf_vector.get_feature_names()[0:11]
xx=tfidf_vector.transform(preprocessed_reviews)
xx.toarray()
print("the type of count vectorizer ",type(xx))
print("the shape of out text TFIDF vectorizer ",xx.get_shape())
print("the number of unique words including both unigrams and bigrams ", xx.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (4986, 12997)
the number of unique words including both unigrams and bigrams  12997


# Word2Vector

In [35]:
list_of_sentance=[]
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())

is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True
if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")
w2v_words=list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])
#print(w2v_model.wv.most_similarity("must","one"))

[('excellent', 0.99699467420578), ('especially', 0.9969537258148193), ('popchips', 0.9968855381011963), ('calorie', 0.9968839287757874), ('healthier', 0.9966698288917542), ('exactly', 0.9966025948524475), ('alternative', 0.9965463280677795), ('cal', 0.9965125322341919), ('snacking', 0.9964659214019775), ('overall', 0.9964284300804138)]
[('wow', 0.9995042085647583), ('de', 0.9994572401046753), ('oatmeal', 0.9994481801986694), ('hands', 0.9994179606437683), ('somewhat', 0.99941086769104), ('varieties', 0.9993906021118164), ('pod', 0.9993851184844971), ('sticks', 0.9993606209754944), ('place', 0.9993605613708496), ('part', 0.9993516802787781)]
number of words that occured minimum 5 times  3817
sample words  ['product', 'available', 'course', 'total', 'pretty', 'stinky', 'right', 'nearby', 'used', 'ca', 'not', 'beat', 'great', 'received', 'shipment', 'could', 'hardly', 'wait', 'try', 'love', 'call', 'instead', 'removed', 'easily', 'daughter', 'designed', 'printed', 'use', 'car', 'windows',

In [37]:
import nltk
nltk.download('stopwords')
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re
paragraph = """I have three visions for India. In 3000 years of our history, people from all over 
               the world have come and invaded us, captured our lands, conquered our minds. 
               From Alexander onwards, the Greeks, the Turks, the Moguls, the Portuguese, the British,
               the French, the Dutch, all of them came and looted us, took over what was ours. 
               Yet we have not done this to any other nation. We have not conquered anyone. 
               We have not grabbed their land, their culture, 
               their history and tried to enforce our way of life on them. 
               Why? Because we respect the freedom of others.That is why my 
               first vision is that of freedom. I believe that India got its first vision of 
               this in 1857, when we started the War of Independence. It is this freedom that
               we must protect and nurture and build on. If we are not free, no one will respect us.
               My second vision for India’s development. For fifty years we have been a developing nation.
               It is time we see ourselves as a developed nation. We are among the top 5 nations of the world
               in terms of GDP. We have a 10 percent growth rate in most areas. Our poverty levels are falling.
               Our achievements are being globally recognised today. Yet we lack the self-confidence to
               see ourselves as a developed nation, self-reliant and self-assured. Isn’t this incorrect?
               I have a third vision. India must stand up to the world. Because I believe that unless India 
               stands up to the world, no one will respect us. Only strength respects strength. We must be 
               strong not only as a military power but also as an economic power. Both must go hand-in-hand. 
               My good fortune was to have worked with three great minds. Dr. Vikram Sarabhai of the Dept. of 
               space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material.
               I was lucky to have worked with all three of them closely and consider this the great opportunity of my life. 
               I see four milestones in my career"""
text = re.sub(r'\[[0-9]*\]',' ',paragraph)
text = re.sub(r'\s+',' ',text)
text = text.lower()
text = re.sub(r'\d',' ',text)
text = re.sub(r'\s+',' ',text)

sentences = nltk.sent_tokenize(text)#separting each lines
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]#seperating lines in word
for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]
    
model = Word2Vec(sentences, min_count=1)
words = model.wv.vocab
print(words)
vector = model.wv['war']
print(vector)
print(len(vector))
similar = model.wv.most_similar('vikram')
print(similar)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'three': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86BE0>, 'visions': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86898>, 'india': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86080>, '.': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86588>, 'years': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A865F8>, 'history': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86710>, ',': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86390>, 'people': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A867F0>, 'world': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86748>, 'come': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A869B0>, 'invaded': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86B70>, 'us': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86C50>, 'captured': <gensim.models.keyedvectors.Vocab object at 0x0000027F27A86EB8>, 'lands': <gensim.models.keyedvectors.Vo


[ 4.0079765e-03 -1.5094128e-04 -4.0452802e-03 -1.6813163e-03
 -4.1083549e-04  4.6132626e-03 -3.4683321e-03 -3.0049069e-03
 -2.0092586e-03 -4.9999598e-03 -4.5861327e-03  3.0940461e-03
  6.6934450e-04  6.1799702e-04 -2.1813685e-04 -1.9340969e-04
 -6.5679179e-04  5.4880203e-04 -3.9964318e-03  5.3768788e-05
 -4.2622038e-03  1.6617109e-03 -2.6905385e-03  1.4107686e-03
  7.4933423e-04  7.1827398e-04 -3.1653068e-03 -1.0338721e-03
 -5.2487239e-04  4.5072706e-03  2.8509167e-03  1.6768486e-03
  4.7445325e-03  3.0656410e-03  4.6061347e-03  4.9838838e-03
 -3.2188697e-03  4.8224628e-03  3.9243191e-03 -1.6997954e-03
  3.0580852e-03  2.5722112e-03  1.9989007e-03 -2.8771502e-03
  2.9010968e-03  1.4361503e-03 -3.7496258e-03  4.3566362e-03
  2.6069521e-03 -3.1477404e-03  3.9159395e-03 -1.3322579e-03
  4.0562921e-03 -4.5983559e-03 -4.5210873e-03 -3.2784941e-04
  4.1620019e-03 -5.5708707e-04  3.6819369e-04 -4.0857177e-03
 -2.9786496e-04 -1.0986668e-03 -4.1592894e-03  2.5099288e-03
 -1.0739318e-03  3.7897

# Average w2v

In [43]:
sent_vectors=[]
for sent in tqdm(list_of_sentance):
    sent_vec=np.zeros(50)
    cnts_words=0
    for word in sent:
        if word in w2v_words:
            vec=w2v_model.wv[word]
            sent_vec+=vec
            cnts_words+=1
    if cnts_words!=0:
        sent_vec/=cnts_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(sent_vectors[0])
print(len(sent_vectors[0]))
            
        

100%|█████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:07<00:00, 679.60it/s]


4986
[ 0.33955976 -0.30691839 -0.10821618  0.14260847  0.21523544 -0.11790287
  0.17669619 -0.29202867  0.00677943 -0.27665481  0.63421599 -0.68676126
  0.07341084  0.01056516  0.38979294 -0.1100363   0.22680589  0.24096573
 -0.11138643  0.18579456 -0.23055225 -0.18485529 -0.2343709  -0.52559459
  0.46168007 -0.56959591 -0.43223304  0.48193788 -0.23224171 -0.13496586
 -0.5050505  -0.01640455 -0.122833   -0.30960927  0.21913188  0.05387778
 -0.57458448 -0.1536215  -0.00927529 -0.44058762  0.11398976  0.60021226
 -0.49233294  0.34059365 -0.45152407  0.56099667 -0.33603182  0.46687454
 -0.03834552 -0.14183397]
50


# tfidf -avg

In [63]:
model = TfidfVectorizer()
model.fit(preprocessed_reviews)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [66]:
tfidf_names=model.get_feature_names()
tfidf_sent_vectors = []
for sent in tqdm(list_of_sentance):
    sent_vec=np.zeros(50)
    yo=0
    for word in sent:
        if word in w2v_words and word in tfidf_names:
            tfidf_values=dictionary[word]*(sent.count(word)/len(sent))
            vec=w2v_model.wv[word]
            aman=tfidf_values*vec
            sent_vec+=aman
            yo+=tfidf_values
    if yo!=0:
        sent_vec/=yo
    tfidf_sent_vectors.append(sent_vec)
print(tfidf_sent_vectors[0])
            

100%|██████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:56<00:00, 87.94it/s]


[ 0.29869608 -0.26596391 -0.08882999  0.11928489  0.19477787 -0.09973884
  0.16864873 -0.24943899  0.00520934 -0.23868851  0.56580343 -0.60710938
  0.06489809  0.01721259  0.34877697 -0.09907576  0.20626144  0.21715222
 -0.09048618  0.17112436 -0.21079553 -0.16902088 -0.20439504 -0.46711158
  0.39874336 -0.4974572  -0.38365502  0.42055098 -0.21274099 -0.12343014
 -0.45321797 -0.01448922 -0.10918763 -0.26461399  0.2021797   0.04056218
 -0.49509727 -0.1367382  -0.00290348 -0.39330259  0.10053372  0.53048221
 -0.43905304  0.30115242 -0.40395149  0.50237181 -0.29324145  0.41274753
 -0.02648397 -0.12262775]
